In [ ]:
import rasterio
import rasterio.plot
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
raster_path = "https://github.com/opengeos/datasets/releases/download/raster/dem_90m.tif"

src = rasterio.open(raster_path)
print(src)

In [ ]:
src.name

In [ ]:
src.mode

In [ ]:
src.meta

In [ ]:
src.crs

In [ ]:
src.res

In [ ]:
src.width

In [ ]:
src.height

In [ ]:
src.bounds

In [ ]:
src.dtypes

In [ ]:
src.transform

In [ ]:
rasterio.plot.show(src)

In [ ]:
rasterio.plot.show(src,1)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
rasterio.plot.show(src, cmap='terrain', ax=ax, title='Digital Elevation Model (DDEM)')
plt.show()  

In [ ]:
dem_bounds ="https://github.com/opengeos/datasets/releases/download/places/dem_bounds.geojson"

gdf = gpd.read_file(dem_bounds)
gdf = gdf.to_crs(src.crs)

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))

rasterio.plot.show(src, cmap='terrain', ax=ax, title='Digital Elevation Model (Dem)')
gdf.plot(ax=ax, edgecolor='blue', facecolor= '#c43c39', lw=2, alpha=0.5)

In [ ]:
red_band =src.read(1)
plt.Figure(figsize=(8,8))
plt.imshow(red_band, cmap='terrain')
plt.colorbar(label="Elevation (meters)",shrink=0.6)
plt.title("DEM with Terrain Colormap")
plt.show()

In [ ]:
raster_path = "https://github.com/opengeos/datasets/releases/download/raster/LC09_039035_20240708_90m.tif"

src = rasterio.open(raster_path)
print(src)

In [ ]:
src.meta

In [ ]:
band_name =["Coastal Aersol", "Blue","Green","Red", "NIR", "SWIR1","SWIR2" ]

In [ ]:
rasterio.plot.show((src, 5), cmap ="Greys_r")

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(8,10))
axes =axes.flatten()

for band in range (1, src.count):
                   data=src.read(band)
                   ax=axes[band-1]
                   im =ax.imshow(data,cmap='gray', vmin=0, vmax=0.5)
                   ax.set_title(f"Band {band_name[band - 1]}")
                   fig.colorbar(im, ax=ax, label="Reflectance", shrink =0.5)

plt.tight_layout()
plt.show()

In [ ]:
nir_band =src.read(5)
red_band =src.read(4)
green_namd =src.read(3)

rgb =np.dstack((nir_band, red_band, green_namd)).clip(0,1)


plt.figure(figsize=(8,8))
plt.imshow(rgb)
plt.title("Bands NIR, Red, Green Combined")
plt.show()

In [ ]:
ndwi = (green_namd-nir_band)/(green_namd+nir_band)
ndwi = ndwi.clip(-1,1)


plt.figure(figsize=(8,8))
plt.imshow(ndwi, cmap='Blues', vmin=-1, vmax=1)
plt.colorbar(label='NDVI', shrink=0.5)
plt.title("NDVI")
plt.xlabel("Column #")
plt.ylabel("Row #")
plt.show()

In [ ]:
ndvi = (nir_band-red_band)/(nir_band+red_band)
ndvi = ndvi.clip(-1,1)


plt.figure(figsize=(8,8))
plt.imshow(ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
plt.colorbar(label='NDVI', shrink=0.5)
plt.title("NDVI")
plt.xlabel("Column #")
plt.ylabel("Row #")
plt.show()

In [ ]:
ndvi.shape

In [ ]:
full_data = src.read()

In [ ]:
full_data.max()

In [ ]:
max_image = np.max(full_data, axis= 0)

In [ ]:
max_image.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(max_image, cmap='RdYlGn', vmin=-0, vmax=0.5)
plt.colorbar(label='NDVI', shrink=0.5)
plt.title("NDVI")
plt.xlabel("Column #")
plt.ylabel("Row #")
plt.show()

In [ ]:
with rasterio.open(raster_path) as src:
    profile = src.profile
print(profile)

In [ ]:
profile.update(count=1)
print(profile)

In [ ]:
output_raster_path = "NDVI.tiff"

with rasterio.open(output_raster_path, "w", **profile) as dst:
    dst.write(ndvi, 1)
print(f"Raster data has been written to {output_raster_path}")

In [ ]:
profile.update(count=3)
print(profile)

In [ ]:
output_raster_path = "rgb.tiff"

with rasterio.open(output_raster_path, "w", **profile) as dst:
    for i in range(1,4):
        dst.write(rgb[:,:,i-1],i)
print(f"Raster data has been written to {output_raster_path}")

In [ ]:
src=rasterio.open(raster_path)
data = src.read()

In [ ]:
data.shape

In [ ]:
subset = data[:,900:1400,700:1200].clip(0,1)
rgb_subset = np.dstack((subset[4],subset[3],subset[2]))
rgb_subset.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(rgb_subset)
plt.title("LAS Vegas, NV")
plt.show()

In [ ]:
from rasterio.windows import Window
from rasterio.transform import from_bounds

window =Window(col_off = 700, row_off = 900, width =500, height = 500)

window_bounds =rasterio.windows.bounds(window, src.transform)

new_transform = from_bounds(*window_bounds, window.width, window.height)

In [ ]:
new_transform

In [ ]:
with rasterio.open(
    "Las_vegas.tiff",
    "w",
    driver="GTiff",
    height=subset.shape[1],
    width=subset.shape[2],
    count=subset.shape[0],
    dtype=subset.dtype,
    crs=src.crs,
    transform=new_transform,
    compress = "lzw",
) as dst:
    dst.write(subset)

In [ ]:

import rasterio.mask

In [ ]:
geojson_path = "https://github.com/opengeos/datasets/releases/download/places/las_vegas_bounds_utm.geojson"
bounds = gpd.read_file(geojson_path)

In [ ]:
fig, ax = plt.subplots()
rasterio.plot.show(src, ax=ax)
bounds.plot(ax=ax, edgecolor='red', facecolor='none')

In [ ]:
import fiona 

with fiona.open(geojson_path, "r") as f:
    shapes = [feature['geometry']] for feature in f

In [ ]:
out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)

In [ ]:
out_meta = src.meta

out_meta.update(
    {
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
    }
)

with rasterio.open("las_vegas_clip.tif", "w", **out_meta) as dst:
    dst.write(out_image)